# IRFs and DL3 scripts

In this notebook, you can try running magic-cta-pipe scripts on a tiny data sample. Because of time constraints, you could not run the pipeline on all the files needed to produce plots (e.g. SED), so we will provide you a complete dataset to get to 'nice' plots and a few MCs and data *.h5* files to run the pipeline.
You have to provide the MCP scripts path and paths (+filenames) where you want to save logging infos.

In [1]:
import glob
import logging
import os
import subprocess
import sys
log = logging.getLogger()

# IRF

Here you will evaluate gammaness and theta cuts and create point-like IRFs (i.e. one angular bin every IRF (ring-wobble gammas); effective area and energy dispersion evaluation)

Options:

-g: input DL2 gamma file (test sample)

-o: output directory (to save IRFs)

-c: configuration file:

1) quality_cuts: "disp_diff_mean < 0.22": standard quality cut on DISPs
2) event_type: "software": hardware=hardware trigger between MAGIC and LST-1; software: any 2-telescopes combination except MAGIC-only; software_only_3tel: only MI + MII +LST-1 combo; magic_only: only MI + MII combo
3) cut_type: "dynamic": global= cut not energy dependent, constant; dynamic: cut evaluated in every energy bin, according to the given efficiency



In [2]:
%%capture
os.system("sed -n '210,254p' /fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml")

create_irf:
    quality_cuts: "disp_diff_mean < 0.22"
    event_type: "software"  # select "software", "software_only_3tel", "magic_only" or "hardware"
    weight_type_dl2: "intensity"  # select "simple", "variance" or "intensity"
    obs_time_irf: "50 h"  # used when creating a background HDU

    energy_bins:  # log space
        start: "0.01 TeV"
        stop: "1000 TeV"
        n_edges: 26

    migration_bins:  # log space
        start: 0.2
        stop: 5
        n_edges: 31

    fov_offset_bins:  # linear space, used for diffuse MCs
        start: "0 deg"
        stop: "1 deg"
        n_edges: 2

    source_offset_bins:  # linear space, used when creating PSF HDU
        start: "0 deg"
        stop: "1 deg"
        n_edges: 101

    bkg_fov_offset_bins:  # linear space, used when creating background HDU
        start: "0 deg"
        stop: "10 deg"
        n_edges: 21

    gammaness:
        cut_type: "dynamic"  # select "global" or "dynamic"
        global_cut_value: 0.8  # use

**Settings**

In [3]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/IRF.log','w')

scripts=('/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_gamma=('/fefs/aswg/workspace/elisa.visentin/sw_school/gamma_2/*.h5')
output_dir_irf=('/fefs/aswg/workspace/elisa.visentin/sw_school/irf/')
config='/fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml'

cd to the scripts directory to launch them

In [4]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from gamma/proton folders 

In [5]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()

Here we use python subprocess.run to run the script and get a log file

In [6]:
for input_file in input_file_gamma: 
    x=subprocess.run(['python','lst1_magic_create_irf.py', f'-g{input_file}', f'-o{output_dir_irf}',\
        f'-c{config}'], stdout=f, stderr=f) 

Here you can check your log file

In [7]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/IRF.log

# DL2 to DL3 data

Here we run a script that interpolates IRFs (to get e.g. gammaness cuts or effective area values for every run average pointing coordinates), produces DL3 data (i.e. DL2 data + gammaness and theta cuts) and packs them toghether with IRFs (*.fits.gz* files). 

Then we run a script to create HDUs and observations index files (needed by Gammapy to process DL3 data and saved in the DL3 data directory)

Options:

-d: input file (DL2 real data)

-i: directory where you saved the IRFs

-o: directory to save DL3 data

-c: configuration file: 
1) interpolation_method: "linear": nearest = use IRFs/cuts value evaluated in the nearest gamma test node; linear/cubic = interpolates IRFs/cuts values by using more than one gamma test node (see also scipy.interpolate)


In [8]:
%%capture
os.system("sed -n '255,265p' /fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml")


dl2_to_dl3:
    interpolation_method: "nearest"  # select "nearest", "linear" or "cubic"
    source_name: "Crab"
    source_ra: null  #"83.633083 deg"  used when the source name cannot be resolved
    source_dec: null  #"22.0145 deg" used when the source name cannot be resolved


**Settings**

In [9]:
f=open('/fefs/aswg/workspace/elisa.visentin/sw_school/DL3.log','w')

input_data=('/fefs/aswg/workspace/elisa.visentin/sw_school/real_2/*.h5')
input_dir_irf=('/fefs/aswg/workspace/elisa.visentin/sw_school/irf/')
output_dir_dl3=('/fefs/aswg/workspace/elisa.visentin/sw_school/dl3')
config=('/fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml')

cd to the scripts directory to launch them

In [10]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Lines to get files from gamma/proton folders 

In [11]:
input_file_data = glob.glob(input_data)
input_file_data.sort()

Here we use python subprocess.run to run the scripts and get a log file

1) DL3 data 

In [12]:
for input_file in input_file_data:  
    y=subprocess.run(['python','lst1_magic_dl2_to_dl3.py', f'-d{input_file}', f'-i{input_dir_irf},\
        'f'-o{output_dir_dl3}', f'-c{config}'], stdout=f, stderr=f)        

2) Index files

In [13]:
z=subprocess.run(['python','lst1_magic_create_irf.py', f'-i{output_dir_dl3}'],stdout=f, stderr=f)        

In [14]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/DL3.log